# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">3. Model Explainability with LIME - Utkarsh Gaikwad</p>

### Introduction to LIME

* [LIME](https://christophm.github.io/interpretable-ml-book/lime.html#lime-for-tabular-data) stands for Local Interpretable Model-agnostic Explanations. LIME focuses on training local surrogate models to explain individual predictions. Local surrogate models are interpretable models that are used to explain individual predictions of black box machine learning models. Surrogate models are trained to approximate the predictions of the underlying black box model. Instead of training a global surrogate model, LIME focuses on training local surrogate models.

* LIME is model-agnostic, meaning that it can be applied to any machine learning model. The technique attempts to understand the model by perturbing the input of data samples and understanding how the predictions change.

![LIME](https://miro.medium.com/v2/resize:fit:1165/1*k-rxjnvUDTwk8Jfg6IYBkQ.png)

* Model-specific approaches aim to understand the black model machine learning model by analysing the internal components and how they interact. LIME provides local model interpretability. LIME modifies a single data sample by tweaking the feature values and observes the resulting impact on the output. The most common question is probably: why was this prediction made or which variables caused the prediction.

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Reading Dataset</p>

In [1]:
import pandas as pd
df = pd.read_csv('./data/gemstone.csv')
df.head()

id  carat        cut color clarity  depth  table     x     y     z  price
0   0   1.52    Premium     F     VS2   62.2   58.0  7.27  7.33  4.55  13619
1   1   2.03  Very Good     J     SI2   62.0   58.0  8.06  8.12  5.05  13387
2   2   0.70      Ideal     G     VS1   61.2   57.0  5.69  5.73  3.50   2772
3   3   0.32      Ideal     G     VS1   61.6   56.0  4.38  4.41  2.71    666
4   4   1.70    Premium     G     VS2   62.6   59.0  7.65  7.61  4.77  14453

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Seperating X and Y</p>

In [2]:
X = df.drop(labels=['id','price'],axis=1)
Y = df[['price']]

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Train Test Split</p>

In [3]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.2,random_state=42)

In [4]:
xtrain.shape

(154858, 9)

In [5]:
xtest.shape

(38715, 9)

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Performing preprocessing</p>

In [6]:
import pickle
with open('E:/Gemstone Price Prediction/artifacts/preprocessor.pkl','rb') as file:
    preprocessor = pickle.load(file)

In [7]:
xtrain_scaled = preprocessor.fit_transform(xtrain)
xtest_scaled = preprocessor.transform(xtest)

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Model Prediction</p>

In [8]:
import pickle
with open('E:/Gemstone Price Prediction/artifacts/model.pkl','rb') as file:
    model = pickle.load(file)

In [9]:
ytrain_pred = model.predict(xtrain_scaled)
ytest_pred = model.predict(xtest_scaled)

# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Model Evaluation</p>

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [11]:
# Evaluate Train and Test dataset
model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(ytrain, ytrain_pred)
model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(ytest, ytest_pred)

# Printing the Evaluation results
print('Model performance for Training set')
print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
print("- R2 Score: {:.4f}".format(model_train_r2))

print('\n----------------------------------\n')
    
print('Model performance for Test set')
print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
print("- R2 Score: {:.4f}".format(model_test_r2))

Model performance for Training set
- Root Mean Squared Error: 535.0317
- Mean Absolute Error: 283.3448
- R2 Score: 0.9824

----------------------------------

Model performance for Test set
- Root Mean Squared Error: 575.3857
- Mean Absolute Error: 294.1008
- R2 Score: 0.9795


# <p style="padding:10px;background-color:#87CEEB ;margin:10;color:#000000;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 10px 10px ;overflow:hidden;font-weight:50">Model Interpretation with LIME</p>

### Create Explainer

In [12]:
from lime.lime_tabular import LimeTabularExplainer
features = list(preprocessor.get_feature_names_out())
explainer = LimeTabularExplainer(xtrain_scaled,
                                 feature_names=list(preprocessor.get_feature_names_out()), 
                                 class_names=['price'],
                                 verbose=True,
                                 mode='regression')

In [13]:
# Choose the 6th instance and use it to predict the results
j = 6
exp = explainer.explain_instance(xtest_scaled[j], model.predict, num_features=9)

Intercept 2548.6769664338653
Prediction_local [8589.21455586]
Right: 6655.7531159335695


In [14]:
# Show the predictions
exp.show_in_notebook(show_table=True)

In [15]:
exp.as_list()

[('num_pipeline__y > 0.72', 2833.8666882261678),
 ('num_pipeline__carat > 0.52', 2483.956087806949),
 ('num_pipeline__x > 0.72', 1164.3928873395807),
 ('num_pipeline__z > 0.72', 1162.2594502879185),
 ('cat_pipeline__clarity <= -0.65', -1112.4409150057938),
 ('0.30 < cat_pipeline__color <= 0.91', -445.70922661083233),
 ('-1.14 < cat_pipeline__cut <= -0.13', -46.50929913472459),
 ('-0.12 < num_pipeline__table <= 0.40', 8.817848131043037),
 ('num_pipeline__depth <= -0.48', -8.095931613430611)]